<a href="https://colab.research.google.com/github/guilhermelaviola/SRTTranslator/blob/main/SRTTranslatorModelNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Subtitle translation model***

In [1]:
pip install transformers sentencepiece srt

  Preparing metadata (setup.py) ... done
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=68fa298fb39e84c56052242db657150bcccf30d7616ac96abf09436335fab2e4
  Stored in directory: /root/.cache/pip/wheels/1f/43/f1/23ee9119497fcb57d9f7046fbf34c6d9027c46a1fa7824cf08
Successfully built srt


In [ ]:
# Logging in to Hugging Face Hub:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [2]:
# Loading a Pretrained Translation Model:
from transformers import MarianMTModel, MarianTokenizer

# Load MarianMT for Spanish to Portuguese
model_name = "Helsinki-NLP/opus-mt-es-pt"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: Helsinki-NLP/opus-mt-es-pt is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# Parsing and translating SRT files:
import srt

def translate_text(text):
    """Translates Spanish text to Brazilian Portuguese."""
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model.generate(**inputs)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def translate_srt(input_srt_path, output_srt_path):
    """Reads, translates, and writes the translated SRT file."""
    with open(input_srt_path, "r", encoding="utf-8") as file:
        subtitles = list(srt.parse(file.read()))

    for subtitle in subtitles:
        subtitle.content = translate_text(subtitle.content)

    # Write translated subtitles to a new file
    with open(output_srt_path, "w", encoding="utf-8") as file:
        file.write(srt.compose(subtitles))

    print(f"Translation completed: {output_srt_path}")

# Example usage
translate_srt("input.srt", "translated.srt")